<a href="https://colab.research.google.com/github/ImVis10/1PercentStrategy/blob/master/1PercentStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv  
import time

In [ ]:
API_KEY = 'OTPBXZ24V7Q9A24S'

In [ ]:
def write_to_csv(stock,data, mode):


  # header = ['time', 'open', 'high', 'low', 'close', 'volume']

  with open("data/"+stock+'_minute.csv', mode, encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(data)

In [ ]:
time_periods ={
    '1':'year1month1',
    '2':'year1month2',
    '3':'year1month3',
    '4':'year1month4',
    '5':'year1month5',
    '6':'year1month6',
    '7':'year1month7',
    '8':'year1month8',
    '9':'year1month9',
    '10':'year1month10',
    '11':'year1month11',
    '12':'year1month12',
    '13':'year2month1',
    '14':'year2month2',
    '15':'year2month3',
    '16':'year2month4',
    '17':'year2month5',
    '18':'year2month6',
    '19':'year2month7',
    '20':'year2month8',
    '21':'year2month9',
    '22':'year2month10',
    '23':'year2month11',
    '24':'year2month12',
}

In [ ]:
import csv
import requests

def get_stock_csv(stock_name):
  # Run only 5 time periods in a minute to not exceed the API hits
  num_calls_in_a_minute = 0
  max_calls_in_a_minute = 5
  for month, time_period in time_periods.items():
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=%(stock)s&interval=1min&slice=%(slice)s&apikey=%(key)s&adjusted=false'\
            %{'stock':stock_name,'slice':time_period,'key':API_KEY}
    with requests.Session() as s:
        if num_calls_in_a_minute == max_calls_in_a_minute:
            num_calls_in_a_minute = 0
            time.sleep(60)
        print(time_period)
        download = s.get(CSV_URL)
        num_calls_in_a_minute += 1
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        for row in my_list:
          if 'open' not in row:
            write_to_csv(stock_name, row,'a')
get_stock_csv('GOOG')          

year1month1
year1month2
year1month3
year1month4
year1month5
year1month6
year1month7
year1month8
year1month9
year1month10
year1month11
year1month12
year2month1
year2month2
year2month3
year2month4
year2month5
year2month6
year2month7
year2month8
year2month9
year2month10
year2month11
year2month12


In [ ]:
def get_stock_data(stock_name):
    mode = 'r'
    interval = 'minute'
    file_path = 'data/' + stock_name+'_' + interval +'.csv'
    minute_data = []
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
                minute_data.append(list(row))
    return minute_data

In [ ]:
TSLA_minute_data = get_stock_data('TSLA')
AAPL_minute_data = get_stock_data('AAPL')
STNG_minute_data = get_stock_data('STNG')
SPY_minute_data = get_stock_data('SPY')


In [ ]:
def get_daily_opening_candle(minute_data):
  daily_opening_candle = []
  date_and_time = minute_data[0][0].split()
  current_date = date_and_time[0]
  for i in range(len(minute_data)):
    date_and_time = minute_data[i][0].split()
    end_date = date_and_time[0]


    # if there is a change in the current_date, get the data
    if (current_date != end_date):
      daily_opening_candle.append(minute_data[i-1])
      current_date = end_date

  daily_opening_candle.append(minute_data[-1])
  daily_opening_candle.reverse()
  return daily_opening_candle

In [ ]:
TSLA_daily_opening_candle = get_daily_opening_candle(TSLA_minute_data)
AAPL_daily_opening_candle = get_daily_opening_candle(AAPL_minute_data)
STNG_daily_opening_candle = get_daily_opening_candle(STNG_minute_data)
SPY_daily_opening_candle = get_daily_opening_candle(SPY_minute_data)



In [ ]:
def get_daily_closing_candle(minute_data):
    daily_closing_candle=[]
    current_date = ''
    for i in range(len(minute_data)):
        date_and_time = minute_data[i][0].split()
        end_date = date_and_time[0]

        # if there is a change in the current_date, get the data
        if (current_date != end_date):
            daily_closing_candle.append(minute_data[i])
            current_date = end_date

    daily_closing_candle.reverse()
    return daily_closing_candle


In [ ]:
TSLA_daily_closing_candle = get_daily_closing_candle(TSLA_minute_data)
AAPL_daily_closing_candle = get_daily_closing_candle(AAPL_minute_data)
STNG_daily_closing_candle = get_daily_closing_candle(STNG_minute_data)
SPY_daily_closing_candle = get_daily_closing_candle(SPY_minute_data)


In [ ]:
# Initialize
PRINCIPAL = 1000
returns = []
num_trades = 0


In [ ]:
def close_trade(gain_or_loss):
    global PRINCIPAL
    global num_trades
    returns.append(gain_or_loss)
    PRINCIPAL += gain_or_loss
    num_trades += 1


In [ ]:
def get_time_series(date, minute_data):

    time_series_data = []

    for i in range(1, len(minute_data) ):
        current_date = minute_data[i][0].split()[0]
        if (current_date == date):
            time_series_data.append(minute_data[i])
            next_date = minute_data[i+1][0].split()[0]
            if (current_date != next_date):
                break
    time_series_data.reverse()
    return time_series_data

In [ ]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bull ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit
def go_bull(profit_percent, buy_price, time_series_data):
    target = buy_price + (buy_price * profit_percent)
    stop_loss = profit_percent/2
    bottom_line = buy_price - (buy_price*stop_loss)
    # Start from second entry because we entered at first entry
    for row in time_series_data[1:]:
        # Check for stop loss first
        #  Comparing with open and low
        if (max(float(row[1]), float(row[3])) <= bottom_line):
            close_trade(max(float(row[1]), float(row[3])) - buy_price)
            return None
        # Comparing with open and high
        if (float(row[1]), float(row[2]) >= target):
            close_trade(max(float(row[1]), float(row[2])) - buy_price)
            return None
    close_trade(buy_price - float(time_series_data[-1][4]))


In [ ]:
# Principal
# Current_principal = Principal - selling_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + selling_price + Gain
########################### Go bear ##################

# Sell
# Wait until you make desired profit or hit the SL
# Exit

def go_bear(profit_percent, sell_price, time_series_data):
    target = sell_price - (sell_price * profit_percent)
    stop_loss = profit_percent/2
    top_line = sell_price + (sell_price*stop_loss)


# Start from second entry because we entered at first entry
    for row in time_series_data[1:]:
        # Check for stop loss first
        #  Comparing with open and high
        if (min(float(row[1]), float(row[2])) >= top_line):
            close_trade(sell_price - min(float(row[1]), float(row[2])))
            return None
        # Comparing with open and low
        if (min(float(row[1]), float(row[3])) <= target):
            close_trade(sell_price - min(float(row[1]), float(row[3])))
            return None
    # Neither conditions hit. Close at the end of the trading day
    close_trade(sell_price - float(time_series_data[-1][4]))


In [ ]:
def run_simulation(profit_percentage, diff_between_trading_sessions, minute_data, daily_opening_candle, daily_closing_candle):

    for i in range(len(daily_opening_candle)-1):

        prev_stock_price = float(daily_closing_candle[i][4])
        stock_price = float(daily_opening_candle[i+1][4])

        bullish_condition = stock_price > prev_stock_price * \
            (1 + diff_between_trading_sessions)
        bearish_condition = stock_price < prev_stock_price * \
            (1 - diff_between_trading_sessions)

        if (bullish_condition):
            time_series = get_time_series(
                daily_opening_candle[i+1][0].split()[0], minute_data)
            go_bull(profit_percentage, stock_price, time_series)

        elif (bearish_condition):
            time_series = get_time_series(
                daily_opening_candle[i+1][0].split()[0], minute_data)
            go_bear(profit_percentage, stock_price, time_series)


# TSLA
1. Profit percentage = 1%
2. Difference between trading sessions = 0.5%

In [ ]:
PRINCIPAL = 1000
num_trades = 0
profit_percentage = 1/100
diff_between_trading_sessions = 0.5/100

run_simulation(profit_percentage, diff_between_trading_sessions,TSLA_minute_data,TSLA_daily_opening_candle, TSLA_daily_closing_candle)
print('Cash in hand: ', PRINCIPAL)
print('Number of days traded in 2 years: ', num_trades)

Cash in hand:  1187.6200000000003
Number of days traded in 2 years:  269


# AAPL
1. Profit percentage = 1%
2. Difference between trading sessions = 0.5%

In [ ]:
PRINCIPAL = 1000
num_trades = 0
profit_percentage = 1/100
diff_between_trading_sessions = 0.5/100

run_simulation(profit_percentage, diff_between_trading_sessions,AAPL_minute_data,AAPL_daily_opening_candle, AAPL_daily_closing_candle)
print('Cash in hand: ', PRINCIPAL)
print('Number of days traded in 2 years: ', num_trades)

IndexError: list index out of range

# STNG
1. Profit percentage = 1%
2. Difference between trading sessions = 0.5%

In [ ]:
PRINCIPAL = 1000
num_trades = 0
profit_percentage = 1/100
diff_between_trading_sessions = 0.5/100

run_simulation(profit_percentage, diff_between_trading_sessions,STNG_minute_data,STNG_daily_opening_candle, STNG_daily_closing_candle)
print('Cash in hand: ', PRINCIPAL)
print('Number of days traded in 2 years: ', num_trades)

Cash in hand:  980.5219999999999
Number of days traded in 2 years:  320


# SPY
1. Profit percentage = 1%
2. Difference between trading sessions = 0.5%

In [ ]:
PRINCIPAL = 1000
num_trades = 0
profit_percentage = 1/100
diff_between_trading_sessions = 0.5/100

run_simulation(profit_percentage, diff_between_trading_sessions,SPY_minute_data,SPY_daily_opening_candle, SPY_daily_closing_candle)
print('Cash in hand: ', PRINCIPAL)
print('Number of days traded in 2 years: ', num_trades)

Cash in hand:  1011.6839999999993
Number of days traded in 2 years:  102
